In [237]:
class Product:
    def __init__(self, product_type: str, product_name: str, product_price: float):
        self.product_type = product_type
        self.product_name = product_name
        self.product_price = product_price

    def __str__(self):
        return f"{self.product_name.capitalize()}({self.product_type}): {self.product_price} UAH"

In [238]:
class MethodError(ValueError):
    def __init__(self, message: str):
        super().__init__(message)
        self.message = message

    def __str__(self):
        return self.message

In [239]:
def error_handling(error: MethodError):
    try:
        raise error
    except MethodError as e:
        print(f'{e.__class__} has been detected.\nThe reason: {e}')

In [240]:
class Catalog:
    def __init__(self):
        self.catalog = {}

    def add_product(self, product: Product, amount: int):
        if product.product_type in self.catalog:
            if product.product_name in self.catalog[product.product_type]:
                if product.product_price in self.catalog[product.product_type][product.product_name]:
                    self.catalog[product.product_type][product.product_name][product.product_price] += amount
                else:
                    self.catalog[product.product_type][product.product_name][product.product_price] = amount
            else:
                self.catalog[product.product_type][product.product_name] = {product.product_price: amount}
        else:
            self.catalog[product.product_type] = {product.product_name: {product.product_price: amount}}

    def remove_product(self, product: Product, amount: int):
        operation = False # Is the operation successfully completed
        message = '' # Message in case if the product isn't in the catalog
        if product.product_type in self.catalog:
            if product.product_name in self.catalog[product.product_type]:
                if product.product_price in self.catalog[product.product_type][product.product_name]:
                    self.catalog[product.product_type][product.product_name][product.product_price] -= amount
                    operation = True
                else:
                    message = 'the product with the price'
            else:
                message = 'the product'
        else:
            message = 'the type of products'
        if not operation:
            final_message = f"Sorry, but we don't have {message} in our catalog."
            error = MethodError(final_message)
            error_handling(error)

    def show_catalog(self):
        print(self.catalog)

In [241]:
class Depository:
    def __init__(self, max_product_numbers: int):
        self.max_product_numbers = max_product_numbers
        self.number_of_products = 0
        self.products = {}

    def add_product(self, product: Product, amount: int):
        if amount + self.number_of_products < self.max_product_numbers:
            if product in self.products:
                self.products[product] += amount
            else:
                self.products[product] = amount
            self.number_of_products += amount
            return True
        else:
            error = MethodError('Sorry, but we cannot add more products in our depository.')
            error_handling(error)
            return False

    def give_product(self, product_name, amount: int):
        message = ''
        for product in self.products:
            if product_name == product.product_name:
                if amount <= self.products[product]:
                    self.products[product] -= amount
                    print(f'Here are your {amount} {product.product_name} for {product.product_price} UAH')
                    return product
                else:
                    message = 'the required amount of '
        final_message = f"Sorry, but we don't have {message}{product_name} in our depository."
        error = MethodError(final_message)
        error_handling(error)
        return False

    def show_products(self):
        for one in self.products:
            print(one)

    def get_information(self):
        information = []
        for our_product in self.products:
            info = our_product.product_name, our_product.product_type, our_product.product_price, self.products[our_product]
            information.append(info)
        return information



In [242]:
class Cashier:
    def __init__(self, cash: float, price_premium: int=30):
        self.start_cash = cash
        self.cash = cash
        self.price_premium = price_premium
        self.bought_products = 0
        self.sold_products = 0
        self.income = 0
        self.discounts = []


    def set_price_premium(self, new_price_premium: int):
        self.price_premium = new_price_premium

    def set_discount(self, identifier: str, discount: int, type_identifier: str):
        new_discount = type_identifier, identifier, discount
        self.discounts.append(new_discount)

    def del_discount(self, identifier: str, discount: int, type_identifier: str):
        old_discount = type_identifier, identifier, discount
        for our_discount in self.discounts:
            if old_discount == our_discount:
                self.discounts.remove(our_discount)

    def show_discounts(self):
        print('Today we have discounts:')
        for discount in self.discounts:
            print(f'\t{discount[1].capitalize()} - {discount[2]}% off')

    def buy_product(self, product: Product, amount: int):
        if product.product_price * amount <= self.cash:
            self.cash -= round(product.product_price * amount, 2)
            self.income = self.cash - self.start_cash
            product.product_price = round(product.product_price * (1 + self.price_premium / 100), 1)
            self.bought_products += amount
            return True
        else:
            final_message = f"We don't have enough money to buy the {product.product_name} in the amount of {amount} pcs"
            error = MethodError(final_message)
            error_handling(error)
            return False

    def sell_product(self, product: Product, amount: int):
        current_price = product.product_price
        for discount in self.discounts:
            if discount[0] == 'type' and discount[1] == product.product_type:
                current_price = round(product.product_price * (1 - discount[2] / 100), 1)
            if discount[0] == 'name' and discount[1] == product.product_name:
                current_price = round(product.product_price * (1 - discount[2] / 100), 1)
        self.cash += current_price * amount
        self.income = round(self.cash - self.start_cash, 1)
        self.sold_products += amount

In [243]:
class ProductStore:
    def __init__(self, store_name: str, depository_volume: int, start_cash: float):
        self.store_name = store_name
        self.catalog = Catalog()
        self.depository = Depository(depository_volume)
        self.cashier = Cashier(start_cash)

    def greeting(self):
        print(f"Welcome to the <<{self.store_name.upper()}>> store!")

    def add_product(self, product: Product, amount: int):
        if self.cashier.buy_product(product, amount) and self.depository.add_product(product, amount):
            self.catalog.add_product(product, amount)

    def sell_product(self, product_name: str, amount: int):
        if result := self.depository.give_product(product_name, amount):
            self.catalog.remove_product(result, amount)
            self.cashier.sell_product(result, amount)

    @staticmethod
    def make_discount_price(our_products: list[tuple], our_discounts: list[tuple]):
        our_products_discount = []
        for our_product in our_products:
            for our_discount in our_discounts:
                our_product = list(our_product)
                if our_product[0] == our_discount[1] or our_product[1] == our_discount[1]:
                    our_product[2] = round(our_product[2] * (1 - our_discount[2] / 100), 1)
            our_products_discount.append(tuple(our_product))
        return set(our_products_discount)

    def get_all_products(self):
        our_discount_products = self.make_information()
        for our_discount_product in our_discount_products:
            if our_discount_product[3] != 0:
                print(f'{our_discount_product[0].title()}({our_discount_product[1]}): {our_discount_product[2]} UAH - {our_discount_product[3]} pieces.')

    def make_information(self):
        our_products = self.depository.get_information()
        our_discounts = self.cashier.discounts
        return self.make_discount_price(our_products, our_discounts)


    def get_product_info(self, name_product):
        our_discount_products = self.make_information()
        for our_discount_product in our_discount_products:
            if our_discount_product[0] == name_product:
                print(f'{our_discount_product[0].title()}({our_discount_product[1]}): {our_discount_product[2]} UAH - {our_discount_product[3]} pieces.')

    def get_income(self):
        return self.cashier.income

    def set_discount(self, identifier: str, discount: int, type_identifier: str):
        self.cashier.set_discount(identifier, discount, type_identifier)

In [244]:
"""PRODUCTS:"""

'PRODUCTS:'

In [245]:
bananas = Product('fruits', 'bananas', 65.5)

In [246]:
oranges = Product('fruits', 'oranges', 70.0)

In [247]:
socks = Product('clothes', 'socks', 25.0)

In [248]:
apples = Product('fruits', 'apples', 10.5)

In [249]:
beetroots = Product('vegetables', 'beetroots', 12.8)

In [250]:
potatoes = Product('vegetables', 'potatoes', 8.0)

In [251]:
t_shirts = Product('clothes', 't_shirts', 150.0)

In [252]:
"""THE <<ATB>> STORE"""

'THE <<ATB>> STORE'

In [253]:
atb = ProductStore('atb', 1000, 50000)

In [254]:
"""LET'S START!"""

"LET'S START!"

In [255]:
atb.greeting()

Welcome to the <<ATB>> store!


In [256]:
"""LET'S BUY PRODUCTS!"""

"LET'S BUY PRODUCTS!"

In [257]:
atb.add_product(bananas, 100)

In [258]:
atb.add_product(oranges, 100)

In [259]:
atb.add_product(socks, 50)

In [260]:
atb.add_product(apples, 200)

In [261]:
atb.add_product(beetroots, 100)

In [262]:
atb.add_product(potatoes, 300)

In [263]:
atb.add_product(t_shirts, 25)

In [264]:
atb.get_all_products()

Apples(fruits): 13.7 UAH - 200 pieces.
Bananas(fruits): 85.2 UAH - 100 pieces.
T_Shirts(clothes): 195.0 UAH - 25 pieces.
Potatoes(vegetables): 10.4 UAH - 300 pieces.
Beetroots(vegetables): 16.6 UAH - 100 pieces.
Socks(clothes): 32.5 UAH - 50 pieces.
Oranges(fruits): 91.0 UAH - 100 pieces.


In [265]:
atb.catalog.show_catalog()

{'fruits': {'bananas': {85.2: 100}, 'oranges': {91.0: 100}, 'apples': {13.7: 200}}, 'clothes': {'socks': {32.5: 50}, 't_shirts': {195.0: 25}}, 'vegetables': {'beetroots': {16.6: 100}, 'potatoes': {10.4: 300}}}


In [266]:
atb.depository.products

{<__main__.Product at 0x1c884b327d0>: 100,
 <__main__.Product at 0x1c884b6aa50>: 100,
 <__main__.Product at 0x1c884b6a710>: 50,
 <__main__.Product at 0x1c884b6a3d0>: 200,
 <__main__.Product at 0x1c884b6ad90>: 100,
 <__main__.Product at 0x1c884b31ad0>: 300,
 <__main__.Product at 0x1c884af8090>: 25}

In [267]:
atb.cashier.cash

25670.0

In [268]:
atb.get_income()

-24330.0

In [269]:
"""LET'S SELL IT!"""

"LET'S SELL IT!"

In [270]:
atb.sell_product("bananas", 50)

Here are your 50 bananas for 85.2 UAH


In [271]:
atb.sell_product('oranges', 30)

Here are your 30 oranges for 91.0 UAH


In [272]:
atb.sell_product('socks', 20)

Here are your 20 socks for 32.5 UAH


In [273]:
atb.sell_product('apples', 120)

Here are your 120 apples for 13.7 UAH


In [274]:
atb.sell_product('beetroots', 60)

Here are your 60 beetroots for 16.6 UAH


In [275]:
atb.sell_product('potatoes', 180)

Here are your 180 potatoes for 10.4 UAH


In [276]:
atb.sell_product('t_shirts', 5)

Here are your 5 t_shirts for 195.0 UAH


In [277]:
atb.get_all_products()

Oranges(fruits): 91.0 UAH - 70 pieces.
Apples(fruits): 13.7 UAH - 80 pieces.
T_Shirts(clothes): 195.0 UAH - 20 pieces.
Potatoes(vegetables): 10.4 UAH - 120 pieces.
Beetroots(vegetables): 16.6 UAH - 40 pieces.
Socks(clothes): 32.5 UAH - 30 pieces.
Bananas(fruits): 85.2 UAH - 50 pieces.


In [278]:
atb.cashier.cash

38797.0

In [279]:
atb.get_product_info('beetroots')

Beetroots(vegetables): 16.6 UAH - 40 pieces.


In [280]:
atb.get_product_info('t_shirts')

T_Shirts(clothes): 195.0 UAH - 20 pieces.


In [281]:
atb.set_discount('clothes', 5, 'type')
atb.set_discount('beetroots', 7, 'name')

In [282]:
atb.cashier.show_discounts()

Today we have discounts:
	Clothes - 5% off
	Beetroots - 7% off


In [283]:
atb.get_product_info('beetroots')

Beetroots(vegetables): 15.4 UAH - 40 pieces.


In [284]:
atb.get_product_info('t_shirts')

T_Shirts(clothes): 185.2 UAH - 20 pieces.


In [285]:
"""LET'S SELL ALL PRODUCTS!"""

"LET'S SELL ALL PRODUCTS!"

In [286]:
atb.sell_product("bananas", 50)
atb.sell_product('oranges', 70)
atb.sell_product('socks', 30)
atb.sell_product('apples', 80)
atb.sell_product('beetroots', 39)
atb.sell_product('potatoes', 120)
atb.sell_product('t_shirts', 20)

Here are your 50 bananas for 85.2 UAH
Here are your 70 oranges for 91.0 UAH
Here are your 30 socks for 32.5 UAH
Here are your 80 apples for 13.7 UAH
Here are your 39 beetroots for 16.6 UAH
Here are your 120 potatoes for 10.4 UAH
Here are your 20 t_shirts for 195.0 UAH


In [287]:
atb.get_all_products()

Beetroots(vegetables): 15.4 UAH - 1 pieces.


In [288]:
atb.sell_product('beetroots', 10)

<class '__main__.MethodError'> has been detected.
The reason: Sorry, but we don't have the required amount of beetroots in our depository.


In [289]:
atb.sell_product('onions', 10)

<class '__main__.MethodError'> has been detected.
The reason: Sorry, but we don't have onions in our depository.


In [290]:
atb.cashier.del_discount('clothes', 5, 'type')

In [291]:
atb.cashier.show_discounts()

Today we have discounts:
	Beetroots - 7% off


In [292]:
atb.get_income()

7002.6

In [293]:
atb.cashier.cash

57002.6

In [294]:
atb.cashier.bought_products

875

In [295]:
atb.cashier.sold_products

874